# Pre trained network
![imagen](https://www.researchgate.net/publication/336874848/figure/fig1/AS:819325225144320@1572353764073/Illustrations-of-transfer-learning-a-neural-network-is-pretrained-on-ImageNet-and.png)

Estas son las arquitecturas de redes neuronales más utilizadas en la comunidad. Para más detalle sobre el funcionamiento de cada red, consultar el [Hands on Machine Learning for Python](https://learning.oreilly.com/library/view/hands-on-machine-learning/9781492032632/ch14.html#cnn_chapter).
* VGG-16
* VGG-19
* Inception V3
* XCeption
* ResNet-50

Las redes se pueden incorporar entrenadas, o sin entrenar.

## ResNet50V2

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
import cv2

from tensorflow.keras.applications.resnet_v2 import ResNet50V2, decode_predictions, preprocess_input

In [2]:
base_model = ResNet50V2(input_shape=(224, 224, 3),
                       include_top=True,
                       weights = "imagenet",
                       classifier_activation='softmax')

In [4]:
base_model.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
_________________________________________________________________________________________

Cargamos algunas imagenes desde local, para ver qué tal funciona la red ResNet50V2 ya entrenada.

In [7]:
import os


def read_data(path):
    X = []

    for file in os.listdir(path):
        image = imread(path + '/' + file)
        smallimage = cv2.resize(image, (224, 224))
        print(path + '/' + file)
        
        X.append(smallimage)

    return np.array(X)
    

x_test = read_data('img')

# Procesar las imagenes tal y como entran en el modelo
x_test = preprocess_input(x_test)
print(x_test.shape)

img/bear-1.jpg
img/Captura2.jpg
img/cat.8016.jpg
img/cat.8037.jpg
img/dog.11856.jpg
img/dog.11857.jpg
img/horse.jpg
img/karate.jpg
img/pizza.jpg
(9, 224, 224, 3)


In [8]:
preds = base_model.predict(x_test)

decodes = decode_predictions(preds, top=5)

for j in decodes:
    print('##################')
    
    for i, decode in enumerate(j):
        print('Predicted:\n', decode[1], decode[2])

##################
Predicted:
 brown_bear 0.99944514
Predicted:
 chow 0.0005413945
Predicted:
 American_black_bear 8.784525e-06
Predicted:
 ice_bear 1.1080914e-06
Predicted:
 howler_monkey 4.4057413e-07
##################
Predicted:
 thimble 0.094853126
Predicted:
 candle 0.090577915
Predicted:
 espresso_maker 0.059212975
Predicted:
 tray 0.05894262
Predicted:
 coffee_mug 0.04447485
##################
Predicted:
 Egyptian_cat 0.6597232
Predicted:
 Siamese_cat 0.15741065
Predicted:
 tiger_cat 0.06259996
Predicted:
 lynx 0.022836657
Predicted:
 tabby 0.018356157
##################
Predicted:
 Egyptian_cat 0.82118624
Predicted:
 lynx 0.1293217
Predicted:
 Siamese_cat 0.021827262
Predicted:
 tabby 0.018324584
Predicted:
 tiger_cat 0.0067972722
##################
Predicted:
 Rottweiler 0.7618954
Predicted:
 Brabancon_griffon 0.07461354
Predicted:
 Staffordshire_bullterrier 0.028572643
Predicted:
 EntleBucher 0.01990426
Predicted:
 Doberman 0.019285155
##################
Predicted:
 collie 0

## VGG16
En este caso vamos a importar la red VGG16, que utilizaremos como red preentrenada y completaremos con una fully connected layer. 

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split

IM_SIZE=64

TRAIN_PATH = 'C:/Users/Daney/Desktop/Archivos/Material/Databases/dogs&cats/mini_train/train'
filenames = os.listdir(TRAIN_PATH)
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    categories.append(category)
    
df = pd.DataFrame({
    'filenames': filenames,
    'category': categories
})

train_df, validate_df = train_test_split(df,
                                         test_size=0.20,
                                         random_state=42)

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

In [10]:
train_df.head()

,filenames,category
0,cat.3337.jpg,cat
1,dog.12039.jpg,dog
2,cat.2542.jpg,cat
3,dog.10283.jpg,dog
4,dog.10384.jpg,dog


In [11]:
from keras.preprocessing.image import ImageDataGenerator


# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
validation_datagen = ImageDataGenerator(rescale = 1.0/255. )

In [12]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    TRAIN_PATH,
                                                    x_col='filenames',
                                                    y_col='category',
                                                    batch_size = 20,
                                                    class_mode = 'binary',
                                                    target_size = (IM_SIZE, IM_SIZE))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_dataframe(validate_df,
                                                              TRAIN_PATH,
                                                              x_col='filenames',
                                                              y_col='category',
                                                              batch_size = 20,
                                                              class_mode = 'binary',
                                                              target_size = (IM_SIZE, IM_SIZE))

Found 5260 validated image filenames belonging to 2 classes.
Found 1316 validated image filenames belonging to 2 classes.


In [17]:
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape = (IM_SIZE, IM_SIZE, 3),
                  include_top=False,
                  weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

    
##### FULLY CONNECTED LAYER #####
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ['acc'])

In [18]:
vgghist = model.fit(train_generator,
                    validation_data = validation_generator,
                    epochs = 5)

Epoch 1/5
263/263 [==============================] - 186s 706ms/step - loss: 0.6761 - acc: 0.6474 - val_loss: 0.4748 - val_acc: 0.7705
Epoch 2/5
263/263 [==============================] - 144s 546ms/step - loss: 0.5359 - acc: 0.7230 - val_loss: 0.4517 - val_acc: 0.7774
Epoch 3/5
263/263 [==============================] - 149s 565ms/step - loss: 0.5129 - acc: 0.7402 - val_loss: 0.4902 - val_acc: 0.7515
Epoch 4/5
263/263 [==============================] - 146s 556ms/step - loss: 0.5011 - acc: 0.7506 - val_loss: 0.4362 - val_acc: 0.8040
Epoch 5/5
263/263 [==============================] - 146s 557ms/step - loss: 0.4861 - acc: 0.7588 - val_loss: 0.5097 - val_acc: 0.7470


### Inception

In [19]:
IM_SIZE = 150

train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    TRAIN_PATH,
                                                    x_col='filenames',
                                                    y_col='category',
                                                    batch_size = 20,
                                                    class_mode = 'binary',
                                                    target_size = (IM_SIZE, IM_SIZE))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_dataframe(validate_df,
                                                              TRAIN_PATH,
                                                              x_col='filenames',
                                                              y_col='category',
                                                              batch_size = 20,
                                                              class_mode = 'binary',
                                                              target_size = (IM_SIZE, IM_SIZE))

Found 5260 validated image filenames belonging to 2 classes.
Found 1316 validated image filenames belonging to 2 classes.


In [20]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

base_model = InceptionV3(input_shape = (IM_SIZE, IM_SIZE, 3),
                        include_top = False,
                        weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

87916544/87910968 [==============================] - 99s 1us/step


In [22]:
x = layers.Flatten()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['acc'])

inc_history = model.fit(train_generator,
                       validation_data = validation_generator,
                       steps_per_epoch = 100,
                       epochs = 1)

100/100 [==============================] - 113s 1s/step - loss: 13.6582 - acc: 0.7527 - val_loss: 0.6529 - val_acc: 0.9316


## ResNet50V2
1. Importar solo capas convolucionales sin entrenar
2. Desarrollar la fully connection layer
3. Usar flow_from_dataframe
4. Entrenar toda la red

In [ ]:
IM_SIZE = 32